In [ ]:
#Step01 -- Initializing Spark

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.0 pyspark-shell'
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession

# We add this line to avoid an error : "Cannot run multiple SparkContexts at once". 
# If there is an existing spark context, we will reuse it instead of creating a new context.
sc = SparkContext.getOrCreate()

# local[*]: run Spark locally with as many working processors as logical cores on your machine.
# In the field of `master`, we use a local server with as many working processors (or threads) as possible (i.e. `local[*]`). 
# If there is no existing spark context, we now create a new context
if (sc is None):
    sc = SparkContext(master="local[*]")
spark = SparkSession(sparkContext=sc).builder.appName("FIT5202 Assignment 1 Part-B") \
        .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/FIT5202.crime") \
        .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/FIT5202.crime") \
        .getOrCreate()

In [ ]:
#Step 02 -- Create a Dataframe
#Renaming Crime_Statistics_SA_2010_present as "Crime"

#In this question I'm using spark session variable to create a dataframe

crimeDataFrame = spark.read.csv('Crime.csv',inferSchema=True, header=True)

In [ ]:
#Step 03 -- Writing into MongoDB with overwrite mode

crimeDataFrame.write.format("com.mongodb.spark.sql.DefaultSource").mode("overwrite").save()

In [ ]:
#Step 04 -- Reading it from MongoDB

newDataFrame = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
newDataFrame.printSchema()

In [ ]:
#Step 05 -- Statistics of numeric and string column

#Describe function to provide the summary of the dataframe

dfDescribe = newDataFrame.describe('Reported Date','Offence Count')
dfDescribe.show()

In [ ]:
#Removing null value from Reported Date in order to use it for further questions
from pyspark.sql.functions import col

newDataFrame=newDataFrame.where(col('Reported Date').isNotNull())
newDataFrame.count()

In [ ]:
#Step 06 - Changing String type to Date using UDF

from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

convertToDate = udf (lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())
newdf = newDataFrame.withColumn('Reported Date',convertToDate(col('Reported Date')))
newdf.show()

In [ ]:
#Step 07 - Preliminary Data Analysis
#1 Number of level 2 offence and list of level 2 offence

lvl2Offence=newdf.select('Offence Level 2 Description').where(col('Offence Level 2 Description').isNotNull()).distinct()
print("Number of Level 2 offences are:",lvl2Offence.count())

lvl2Offence.show()

In [ ]:
#Step 07 - Preliminary Data Analysis
#2 Number of offences against person

personOffence=newdf.select('Offence Count')\
.where(col('Offence Level 1 Description') == 'OFFENCES AGAINST THE PERSON')\
.groupBy().sum()\
.withColumnRenamed('sum(Offence Count)','Sum')\
.show()

In [ ]:
#Step 07 - Preliminary Data Analysis
#3 Number of serial criminal tresspasses with more than 1 offence count

trespassCount=newdf.filter((col('Offence Level 2 Description') == 'SERIOUS CRIMINAL TRESPASS') & (col('Offence Count') > 1))
print("Number of serial criminal tresspasses with more than 1 offence count is:",trespassCount.count())

In [ ]:
#Step 07 - Preliminary Data Analysis
#4 percentage of crimes against property

totalCount=newDataFrame.count()
propertyOffence=newdf.filter(col('Offence Level 1 Description') == 'OFFENCES AGAINST PROPERTY').count()
print("Percentage of crimes that are against property is: {}%".format(round(((propertyOffence/totalCount)*100),2)))

In [ ]:
#Step 8.1 - Exploratory data analysis
#1 Crimes per year
# using year function to extract the year from date

from pyspark.sql.functions import year
import matplotlib.pyplot as plt
%matplotlib inline
bar_width = 0.5

yearList=('2010','2011','2012','2013','2014','2015','2016','2017','2018','2019')
crimesPerYear=newdf.select(year('Reported Date').alias('Year'),'Offence Count')\
.groupBy('Year').sum()\
.orderBy('Year').collect()

crimeYear= [row['Year'] for row in crimesPerYear]
crimeCount= [row['sum(Offence Count)'] for row in crimesPerYear]

plt.subplots(figsize=(10,5))
plt.style.use('ggplot')
plt.bar(crimeYear,crimeCount,bar_width, align='center', color='C0')
plt.xticks(crimeYear,yearList)
plt.xlabel('Year')
plt.ylabel('Count')
plt.title('CRIMES PER YEAR')

## 8. 1 ANALYSIS

The bar chart illustrates number of crimes happened between 2010-2019. It can be seen that number of crimes spiked
between 2010 and 2011,however crimes started to dwindle from 2011 to 2014. In the year of 2015 and 2016, number of crimes recorded are almost equal and then the numbers started to decline in heavy margin after 2018.

In [ ]:
#Step 8.2 - Exploratory data analysis
#2  Crimes per month
# using month function to extract the month from date

from pyspark.sql.functions import month
%matplotlib inline
bar_width = 0.5

crimesPerMonth=newdf.select(month('Reported Date').alias('Month'),'Offence Count')\
.groupBy('Month').sum('Offence Count').orderBy('Month')\
.collect()

monthsList=('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')
#crimeMonth=np.arange(len(monthsList))
crimeMonth= [row['Month'] for row in crimesPerMonth]
crimeTotalCount= [row['sum(Offence Count)'] for row in crimesPerMonth]

plt.subplots(figsize=(10,5))
plt.bar(crimeMonth,crimeTotalCount,bar_width, align='center', color='C0')
plt.xticks(crimeMonth,monthsList)
plt.xlabel('Month')
plt.ylabel('Count')
plt.title('CRIMES PER MONTH')
plt.show()

## 8.2 ANALYSIS

The bar chart illustrates number of crimes happened over a period of 12 months.We can see in the month of January and 
October crimes are crossing 70,000 mark, on the other hand April and June recorded lowest crime rates ever. The pattern
what we observe from this bar chart is that crimes are distributed in all 12 months. 

In [ ]:
#Step 8.3 - Exploratory data analysis
#3 Top 20 suburbs with high Crime rates
# using concat function to merge two columns(Suburb and Postcode)

from pyspark.sql.functions import concat,lit
%matplotlib inline

crimesPerSuburb=newdf.select(concat(col('Suburb - Incident'), lit('-'), col('Postcode - Incident')).alias('Suburb'),'Offence Count')\
.where(col('Suburb - Incident').isNotNull())\
.groupBy('Suburb').sum()\
.sort('sum(Offence Count)', ascending=False)\
.take(20)

crimeSuburb= [row['Suburb'] for row in crimesPerSuburb]
crimeSuburbCount= [row['sum(Offence Count)'] for row in crimesPerSuburb]

plt.subplots(figsize=(10,5))
plt.bar(crimeSuburb,crimeSuburbCount,bar_width, align='center', color='C0')
plt.xticks(rotation=45, ha="right")
plt.xlabel('Suburb')
plt.ylabel('Count')
plt.title('CRIMES PER SUBURB')
plt.show()

## 8.3 ANALYSIS

The bar chart illustrates top 20 suburbs with high crime rates. Adelaide suburb with postcode 5000 has the highest
number of crimes reported with 50k approximately while other suburbs has maintained consistency of not crossing 15k mark. Data skewness can be observed, but it's negligible. 

In [ ]:
#Step 8.4 - Exploratory data analysis
#4 SERIOUS CRIMINAL TRESPASS per day and month
# using dayofweek function to find the day crime was recorded

import numpy as np
from pyspark.sql.functions import dayofweek
%matplotlib inline

width=0.5

#Criminal count for SERIOUS CRIMINAL TRESPASS per day

crimesPerDay = newdf.select(dayofweek('Reported Date').alias('Day'),'Offence Level 2 Description','Offence Count')\
.where(col('Offence Level 2 Description') == 'SERIOUS CRIMINAL TRESPASS')\
.groupBy('Day').sum()\
.orderBy('Day')\
.collect()


crimeDay = [row['Day'] for row in crimesPerDay]
crimeDayCount = [row['sum(Offence Count)'] for row in crimesPerDay]
y_crimepos = np.arange(len(crimeDay))

#Criminal count for SERIOUS CRIMINAL TRESPASS per month

tresspassPerMonth = newdf.select(month('Reported Date').alias('Month'),'Offence Level 2 Description','Offence Count')\
.where(col('Offence Level 2 Description') == 'SERIOUS CRIMINAL TRESPASS')\
.groupBy('Month').sum()\
.orderBy('Month')\
.collect()

tresspassMonth = [row['Month'] for row in tresspassPerMonth]
tresspassMonthCount = [row['sum(Offence Count)'] for row in tresspassPerMonth]
y_pos = np.arange(len(tresspassMonth))

#Plotting graphs

plt.subplots(figsize=(10,5))
plt.bar(y_crimepos + 1, crimeDayCount, bar_width, align='center', color='blue',label='DAY')
plt.bar(y_pos + 1 + width, tresspassMonthCount, bar_width, align='center', color='red',label='MONTH')
plt.xlabel('Day & Month')
plt.ylabel('Count')
plt.title('SERIOUS CRIMINAL TRESPASSES PER DAY AND MONTH')
plt.legend(loc='best')
plt.show()

## 8.4 ANALYSIS

The bar chart indicates number of serious criminal trespasses per day and month from 2010 to 2019.Blue bars indicates days of the week starting from sunday to saturday and red bars indicating months from january to december. 
It can be noticed that tuesdays had several crimes compared to other days. Similary the number of crimes recorded in
January and October are huge. From my observation, high crimes are committed in the months of January,March, October 
especially on tuesdays. 